# Context manager
---

```python
with open('some_file') as data_file:
    pass
```

## Under the hood:

```python
with ctx() as x:
    pass
```

Looks like this:


```python
x = ctx().__enter__()
try:
    pass
finally:
    x.__exit__()
```

## Example
---

In [1]:
from sqlite3 import connect

class temptable():
    def __init__(self, cur):
        self._cursor = cur

    def __enter__(self):
        print('Creating table "points"')
        self._cursor.execute('create table points(x int, y int)')
    
    def __exit__(self, *args):
        print('Removing table "points"')
        self._cursor.execute('drop table points')


with connect(":memory:") as conn:
    cursor = conn.cursor()
    with temptable(cursor):
        cursor.execute('insert into points (x, y) values (1, 1)')
        cursor.execute('insert into points (x, y) values (2, 2)')
        
        for row in cursor.execute('select * from points'):
            print(row)

Creating table "points"
(1, 1)
(2, 2)
Removing table "points"


## But how to make sure that the `__exit__` will not be called before `__enter__`
---

### Generators

In [2]:
def temptable(cursor):
    print('Creating table "points"')
    cursor.execute('create table points(x int, y int)')
    yield
    print('Removing table "points"')
    cursor.execute('drop table points')
    
class contextmanager():
    def __init__(self, cur):
        self._cursor = cur

    def __enter__(self):
        self.gen = temptable(self._cursor)
        next(self.gen)
    
    def __exit__(self, *args):
        next(self.gen, None)


with connect(":memory:") as conn:
    cursor = conn.cursor()
    with contextmanager(cursor):
        cursor.execute('insert into points (x, y) values (1, 1)')
        cursor.execute('insert into points (x, y) values (2, 2)')
        
        for row in cursor.execute('select * from points'):
            print(row)

Creating table "points"
(1, 1)
(2, 2)
Removing table "points"


## Make a generic contextmanager
---

In [3]:
class contextmanager():
    def __init__(self, generator):
        self._generator = generator
    
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self

    def __enter__(self):
        self._generator = self._generator(*self.args, **self.kwargs)
        next(self._generator)
    
    def __exit__(self, *args):
        next(self._generator, None)

def temptable(cursor):
    print('Creating table "points"')
    cursor.execute('create table points(x int, y int)')
    yield
    print('Removing table "points"')
    cursor.execute('drop table points')
    
temptable = contextmanager(temptable)

with connect(":memory:") as conn:
    cursor = conn.cursor()
    with temptable(cursor):
        cursor.execute('insert into points (x, y) values (1, 1)')
        cursor.execute('insert into points (x, y) values (2, 2)')
        
        for row in cursor.execute('select * from points'):
            print(row)

Creating table "points"
(1, 1)
(2, 2)
Removing table "points"


## Use a standard contextmanager
---

In [4]:
from contextlib import contextmanager

@contextmanager
def temptable(cursor):
    print('Creating table "points"')
    cursor.execute('create table points(x int, y int)')
    yield
    print('Removing table "points"')
    cursor.execute('drop table points')

with connect(":memory:") as conn:
    cursor = conn.cursor()
    with temptable(cursor):
        cursor.execute('insert into points (x, y) values (1, 1)')
        cursor.execute('insert into points (x, y) values (2, 2)')
        
        for row in cursor.execute('select * from points'):
            print(row)

Creating table "points"
(1, 1)
(2, 2)
Removing table "points"
